# Measuring analog input using the System Management IP
---

## Aim

* This notebook illustrates how to measure the voltage of a unipolar analog signal connected to J14 header.

## Revision History

* Initial Release

---

## Load _base_ Overlay

In [1]:
from time import sleep
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections</h4>
    <li>Connect analog voltages to A1 (Vaux15) and A0 (VAux14). Analog input should be unipolar between 0 and 3.3V.  Make sure AGND pin is connected to the analog source ground.</li>
    </ul>
</div>

### Instantiate the System Management object

In [2]:
xadc=base.system_management_wiz_0.mmio

### Define register offsets

Since there are a number of registers required to measure the analog voltages, define them first.

In [3]:
# define register offsets, masks related to the System Management IP
XSM_SRR_OFFSET=0
XSM_SR_OFFSET=4
XSM_IP_OFFSET=0x200

XSM_AUX14_OFFSET=XSM_IP_OFFSET + 0x278
XSM_AUX15_OFFSET=XSM_IP_OFFSET + 0x27C

XSM_CFR0_OFFSET=XSM_IP_OFFSET + 0x300
XSM_CFR1_OFFSET=XSM_IP_OFFSET + 0x304
XSM_CFR2_OFFSET=XSM_IP_OFFSET + 0x308
XSM_SEQ00_OFFSET=XSM_IP_OFFSET + 0x320
XSM_SEQ01_OFFSET=XSM_IP_OFFSET + 0x324
XSM_SEQ02_OFFSET=XSM_IP_OFFSET + 0x328
XSM_SEQ03_OFFSET=XSM_IP_OFFSET + 0x32C
XSM_SEQ05_OFFSET=XSM_IP_OFFSET + 0x334
XSM_SEQ07_OFFSET=XSM_IP_OFFSET + 0x33c

XSM_CFR0_AVG_VALID_MASK=0x3000
XSM_CFR0_AVG16_MASK=0x1000
XSM_CFR1_SEQ_VALID_MASK=0xF000
XSM_CFR1_SEQ_SAFEMODE_MASK=0x0000
XSM_CFR1_SEQ_CONTINPASS_MASK=0x2000
XSM_SRR_IPRST_MASK=0x0000000A
XSM_SR_EOS_MASK=0x00000040

XSM_SEQ_CH_AUX14=0x40000000
XSM_SEQ_CH_AUX15=0x80000000
XSM_SEQ_CH_AUX_SHIFT=16
XSM_CFR2_CD_SHIFT=8

### Perform measurements and display results

Reset the IP and configure to measure the analog voltages.

In [4]:
# Reset the IP
xadc.write(XSM_SRR_OFFSET,XSM_SRR_IPRST_MASK)
# disable channel sequencer before configuring the sequence register
regvalue=xadc.read(XSM_CFR1_OFFSET)
xadc.write(XSM_CFR0_OFFSET, (regvalue & ~XSM_CFR1_SEQ_VALID_MASK | XSM_CFR1_SEQ_SAFEMODE_MASK))

In [5]:
# Setup the Averaging to be done for the channels in the Configuration 0 register as 16 samples
regvalue=xadc.read(XSM_CFR0_OFFSET)
xadc.write(XSM_CFR0_OFFSET, (regvalue & ~XSM_CFR0_AVG_VALID_MASK | XSM_CFR0_AVG16_MASK))

In [6]:
# Set the Acquisition time for the specified channels
xadc.write(XSM_SEQ07_OFFSET,(XSM_SEQ_CH_AUX14 | XSM_SEQ_CH_AUX15) >> XSM_SEQ_CH_AUX_SHIFT)

In [7]:
# Enable the averaging on the following channels in the Sequencer registers:
# 14th Auxiliary Channel, 15th Auxiliary Channel
xadc.write(XSM_SEQ03_OFFSET, (XSM_SEQ_CH_AUX14 | XSM_SEQ_CH_AUX15) >> XSM_SEQ_CH_AUX_SHIFT)

In [8]:
# Enable the following channels in the Sequencer registers:
# 14th Auxiliary Channel, 15th Auxiliary Channel
xadc.write(XSM_SEQ01_OFFSET, (XSM_SEQ_CH_AUX14 | XSM_SEQ_CH_AUX15) >> XSM_SEQ_CH_AUX_SHIFT)

In [9]:
# Set the ADCCLK frequency equal to 1/32 of System clock for the System Monitor/ADC in the Configuration Register 2
xadc.write(XSM_CFR2_OFFSET,32 << XSM_CFR2_CD_SHIFT)
regvalue=xadc.read(XSM_CFR1_OFFSET)
xadc.write(XSM_CFR0_OFFSET, (regvalue & ~XSM_CFR1_SEQ_VALID_MASK | XSM_CFR1_SEQ_CONTINPASS_MASK))

Wait till the setup is completed, indicated by end of sequence, and then read the voltages at 2 seconds interval.

In [10]:
# Wait till the End of Sequence occurs
xadc.read(XSM_SR_OFFSET)
while ((xadc.read(XSM_SR_OFFSET) & XSM_SR_EOS_MASK) != XSM_SR_EOS_MASK):
    continue

In [12]:
# Read the current value of Aux Channel 14 and 15
for i in range(5):
    print("Channel 14 voltage: "+"{:.3f}".format((xadc.read(XSM_AUX14_OFFSET))*3.3/65536)," V")
    print("Channel 15 voltage: "+"{:.3f}".format((xadc.read(XSM_AUX15_OFFSET))*3.3/65536)," V")
    sleep(2)

Channel 14 voltage: 0.531  V
Channel 15 voltage: 3.210  V
Channel 14 voltage: 0.529  V
Channel 15 voltage: 3.208  V
Channel 14 voltage: 0.528  V
Channel 15 voltage: 3.208  V
Channel 14 voltage: 0.529  V
Channel 15 voltage: 3.208  V
Channel 14 voltage: 0.529  V
Channel 15 voltage: 3.207  V


Copyright (C) 2021 Xilinx, Inc

---

---